<a href="https://colab.research.google.com/github/pedro162/machine_learn/blob/main/reinforcement_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import gym
import random

In [30]:
env = gym.make("Taxi-v3").env

In [31]:
env.reset()
env.render()

In [32]:
#0=south 1=north 2=east 3=west 4=pickup 5=dropoff
print(env.action_space)

Discrete(6)


In [33]:

5*5*5*4

500

In [34]:
# Destination
print(env.observation_space)

Discrete(500)


In [35]:
len(env.P)

500

In [36]:
env.P[484]

{0: [(1.0, 484, -1, False)],
 1: [(1.0, 384, -1, False)],
 2: [(1.0, 484, -1, False)],
 3: [(1.0, 464, -1, False)],
 4: [(1.0, 484, -10, False)],
 5: [(1.0, 484, -10, False)]}

###Training

Diferença temporal (Q-learning)
### **Qt(s,a) = Qt-1(s,a) + @TDt(a,s)**

In [37]:
%%time
from IPython.display import clear_output
import numpy as np
import random
import math
import time

CPU times: user 18 µs, sys: 1 µs, total: 19 µs
Wall time: 22.6 µs


In [38]:
#1-10% 3-90%
#Exploration / exploitation

In [39]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [40]:
q_table[0]

array([0., 0., 0., 0., 0., 0.])

In [41]:
np.argmax(np.array([3, 5]))

1

In [42]:
alpha=0.1
gamma=0.6
epsilon=0.1

for i in range(1000000):
  state = env.reset()
  penalties, rewording = 0,0
  done = False

  while not done:
    #Exploration
    if random.uniform(0,1) < epsilon:
      action = env.action_space.sample()
    else:#Exploitation
      action = np.argmax(q_table[state])

    next_state, reward, done, info = env.step(action)

    q_old = q_table[state, action]
    next_max = np.max(q_table[next_state])

    q_table[state, action] = (1-alpha)*q_old + alpha*(reward + gamma*next_max)

    if reward == -10:
      penalties += 1

    state = next_state

  if i % 100 == 0:
    clear_output(wait=True)
    print(f"Episode: {i}")
print("Training finished")

Episode: 999900
Training finished


In [43]:
q_table[424]

array([ -2.49618789,  -2.4936525 ,  -2.4936525 ,  -2.49619027,
       -11.49122449, -11.49320373])

In [44]:
env.reset()
env.render()

/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


In [45]:
env.encode(2,1,0,3)

223

In [47]:
env.step(1)
env.render()

/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


In [49]:
env.step(1)
env.render()

In [50]:
total_penalty = 0
episodes = 50
frames =  []

for _ in range(episodes):
  state = env.reset()
  penalties, reward = 0,0
  done = False

  while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)

    if reward == -10:
      penalties += 1

    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
    })
  total_penalty += penalties

print(f"Episodes: {episodes}")
print(f"Penalties: {total_penalty}")

Episodes: 50
Penalties: 0


In [51]:
frames[0]

{'frame': '+---------+\n|\x1bR\x1b: | : :G|\n| : | : : |\n| : : : : |\n| |\x1b \x1b: | : |\n|\x1bY\x1b| : |B: |\n+---------+\n  (North)\n',
 'state': 322,
 'action': 1,
 'reward': -1}

In [53]:
for frame in frames:
  clear_output(wait=True)
  print(frame['frame'])
  print(f"State: {frame['state']}")
  print(f"Action: {frame['action']}")
  print(f"Reward: {frame['reward']}")
  time.sleep(.5)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

State: 410
Action: 5
Reward: 20
